In [1]:
import re
import json
import polars as pl

In [29]:
data_df = pl.read_csv('~/sports-language-in-politics/data/raw/political_comments_temporal/sample_2016_2021_political_comments.000000000005.csv').drop_nulls()

In [30]:
data_df.head()

id,link_id,parent_id,created_utc,subreddit,category,super_category,score,body,author,gilded
str,str,str,i64,str,str,str,i64,str,str,i64
"""fmpjipt""","""t3_fwcpyn""","""t3_fwcpyn""",1586281961,"""AOC""","""politics_2019""","""politics""",0,"""The Republican…","""Oikos2""",0
"""f2rfw9o""","""t3_ddure8""","""t3_ddure8""",1570372715,"""law""","""politics_2019""","""politics""",0,"""This is Americ…","""LUEnitedNation…",0
"""eqsphrg""","""t3_bzdsgp""","""t1_eqsgccx""",1560290436,"""law""","""politics_2019""","""politics""",0,"""&gt;But machin…","""boopbaboop""",0
"""f96epyt""","""t3_e3mbwn""","""t1_f93zz05""",1575144007,"""law""","""politics_2019""","""politics""",0,"""Highly unlikel…","""meisannpc""",0
"""g2jdrrq""","""t3_idz5hh""","""t1_g2jdibd""",1598144291,"""law""","""politics_2019""","""politics""",0,"""No, that’s a p…","""King_Posner""",0


In [34]:
data_df["created_utc"].cast(pl.Datetime).dt.with_time_unit('ms')

created_utc
datetime[ms]
1970-01-19 08:38:01.961
1970-01-19 04:12:52.715
1970-01-19 01:24:50.436
1970-01-19 05:32:24.007
1970-01-19 11:55:44.291
1970-01-19 06:55:40.994
1970-01-19 22:46:11.241
1970-01-18 15:26:25.043
1970-01-19 16:16:03.065


In [32]:
data_df.with_columns(
       pl.from_epoch("created_utc", time_unit="ms")
   )

id,link_id,parent_id,created_utc,subreddit,category,super_category,score,body,author,gilded
str,str,str,datetime[ms],str,str,str,i64,str,str,i64
"""fmpjipt""","""t3_fwcpyn""","""t3_fwcpyn""",1970-01-19 08:38:01.961,"""AOC""","""politics_2019""","""politics""",0,"""The Republican…","""Oikos2""",0
"""f2rfw9o""","""t3_ddure8""","""t3_ddure8""",1970-01-19 04:12:52.715,"""law""","""politics_2019""","""politics""",0,"""This is Americ…","""LUEnitedNation…",0
"""eqsphrg""","""t3_bzdsgp""","""t1_eqsgccx""",1970-01-19 01:24:50.436,"""law""","""politics_2019""","""politics""",0,"""&gt;But machin…","""boopbaboop""",0
"""f96epyt""","""t3_e3mbwn""","""t1_f93zz05""",1970-01-19 05:32:24.007,"""law""","""politics_2019""","""politics""",0,"""Highly unlikel…","""meisannpc""",0
"""g2jdrrq""","""t3_idz5hh""","""t1_g2jdibd""",1970-01-19 11:55:44.291,"""law""","""politics_2019""","""politics""",0,"""No, that’s a p…","""King_Posner""",0
"""ffqq2ic""","""t3_euev44""","""t1_ffqnih8""",1970-01-19 06:55:40.994,"""law""","""politics_2019""","""politics""",0,"""Then what did …","""bac5665""",0
"""hl0hmzl""","""t3_qw230i""","""t1_hl0hadl""",1970-01-19 22:46:11.241,"""law""","""politics_2019""","""politics""",0,"""It's actually …","""Thereelgerg""",0
"""dxrrpgy""","""t3_8e1zca""","""t3_8e1zca""",1970-01-18 15:26:25.043,"""law""","""politics_2019""","""politics""",0,"""Oh, apparently…","""deptofgreatjus…",0
"""go1fues""","""t3_lnf7zc""","""t1_go1cf9l""",1970-01-19 16:16:03.065,"""law""","""politics_2019""","""politics""",0,"""I still don’t …","""Runforsecond""",0
